In [45]:
from scipy.io import loadmat
from scipy.stats import zscore
from decimal import *
import pandas as pd
import numpy as np
import datetime
import os
import sys
import math
import pandas as pd
import numpy as np
from pprint import pprint
import json
import requests
import datetime
from datetime import date
import os

In [46]:
# things to change with each person's data
# the df in df.iterrows 

In [47]:
# setting name for node and tag files, which person's dataframe

# distance between each object on the level 1 rods for level 2 objects
space = .00442

In [48]:
pubdata = loadmat('PubData.mat', struct_as_record = True, squeeze_me = True)

In [49]:
pubdata.keys()

dict_keys(['__header__', '__version__', '__globals__', 'Cases50'])

In [50]:
pubdata = {k:v for k, v in pubdata.items() if k[0] != '_'}

In [51]:
# duration of symptoms
a = pubdata['Cases50'][0,1]
b = pubdata['Cases50'][4,1]
c = pubdata['Cases50'][2,1]
d = pubdata['Cases50'][5,1]
#print(a, b, c, d)

#for i in range(0, 50, 1):
#    dtion = pubdata['Cases50'][i,1]
#    print(str('Duration of person ') + str(i) + str(': ') + str(dtion))

In [52]:
cols = ('temp', 'hr', 'hrv', 'breath')

### Version info

In [53]:
#v1: tighter cube of data: -50 to 50
#v2: wider cube of data: -100 to 100 on x and y axis/ z axis up to 225
#v3: change scale min and max of sphere// make x: hr/ y: breath/ z: temp
#v4: same base axes as v3. changed colors so even baseline is full color spectrum
#v5: base axes back to v1 and v2, ie same as Smarr paper zscore viz. full color spectrum for all data points
        #limited to days 42 to 48 of data/
#v5.1: changing scale of spheres

In [54]:
dfos1 = pd.DataFrame(pubdata['Cases50'][5,0], columns = cols)
print(dfos1.shape)

# person variable for file names
person = str("p6")

# version variable for file name
version = "v5.1"

# viz type: 2 days, 2 weeks, or 4 weeks
viztype = "zscore"

# rolling average code
# df = df.rolling(2).mean() 
# df = df.iloc[::2, :]

(93600, 4)


In [55]:
dfos1 = dfos1.iloc[60480:69120,:]
dfos1.index = np.arange(len(dfos1))
print(dfos1.shape)
print(dfos1.head())

(8640, 4)
    temp    hr   hrv  breath
0  30.28  67.0  62.0  20.250
1  30.43  67.0  63.0  19.875
2  30.66  66.8  61.2  20.750
3  30.88  66.6  59.4  17.875
4  31.18  66.4  57.6  20.250


In [56]:
print(dfos1['temp'].min())
print(dfos1['temp'].max())

16.16
37.09


In [57]:
dfos1[['hr_zscore', 'hrv_zscore']] = dfos1[['hr', 'hrv']].apply(zscore)
print(dfos1.head())

    temp    hr   hrv  breath  hr_zscore  hrv_zscore
0  30.28  67.0  62.0  20.250   0.629982    0.719576
1  30.43  67.0  63.0  19.875   0.629982    0.756915
2  30.66  66.8  61.2  20.750   0.601439    0.689704
3  30.88  66.6  59.4  17.875   0.572896    0.622494
4  31.18  66.4  57.6  20.250   0.544353    0.555283


In [58]:
# creating z axis data with length of index

zrange = len(dfos1.index)
zr = (250/zrange)
z = np.arange(1.5, 251.5, zr)
dfos1['timeaxis'] = z
print(dfos1.head())

    temp    hr   hrv  breath  hr_zscore  hrv_zscore  timeaxis
0  30.28  67.0  62.0  20.250   0.629982    0.719576  1.500000
1  30.43  67.0  63.0  19.875   0.629982    0.756915  1.528935
2  30.66  66.8  61.2  20.750   0.601439    0.689704  1.557870
3  30.88  66.6  59.4  17.875   0.572896    0.622494  1.586806
4  31.18  66.4  57.6  20.250   0.544353    0.555283  1.615741


In [59]:
print(dfos1['temp'].mean())

30.940965277777785


dfos5 = pd.DataFrame(pubdata['Cases50'][4,0], columns = cols)
print(dfos5.shape)
dfos3 = pd.DataFrame(pubdata['Cases50'][2,0], columns = cols)
dfos3.shape
dfos6 = pd.DataFrame(pubdata['Cases50'][5,0], columns = cols)
dfos6.shape

## Scaling raw data

In [60]:
def make_interpolater(left_min, left_max, right_min, right_max): 
    # Figure out how 'wide' each range is  
    leftSpan = left_max - left_min  
    rightSpan = right_max - right_min  

    # Compute the scale factor between left and right values 
    scaleFactor = float(rightSpan) / float(leftSpan) 

    # create interpolation function using pre-calculated scaleFactor
    def interp_fn(value):
        return right_min + (value-left_min)*scaleFactor


    return interp_fn

In [61]:
col_min = dfos1['breath'].min() 
col_max = dfos1['breath'].max() 
col_list = dfos1['breath'].tolist()
scalar = make_interpolater(col_min +.000000002, col_max, .8, 1.9) 
col_interp = [scalar(x) for x in col_list] 
dfos1['breath_scale'] = col_interp

In [62]:
col_min = dfos1['hr_zscore'].min() 
col_max = dfos1['hr_zscore'].max() 
col_list = dfos1['hr_zscore'].tolist()
scalar = make_interpolater(col_min, col_max, -200, 200) 
col_interp = [scalar(x) for x in col_list] 
dfos1['hr_scale'] = col_interp

In [63]:
col_min = dfos1['hrv_zscore'].min() 
col_max = dfos1['hrv_zscore'].max() 
col_list = dfos1['hrv_zscore'].tolist()
scalar = make_interpolater(col_min, col_max, -200, 200) 
col_interp = [scalar(x) for x in col_list] 
dfos1['hrv_scale'] = col_interp

In [64]:
# set DEBUG flag
DEBUG = 0

# Initialize all ANTz Node variables
# You shouldn't need to change anything in this section
# you can find an explanation of these terms in Appendix B of the user manual
id = 0
type = 5
data = id
selected = 0
parent_id = 0
branch_level = 0
child_id = id
child_index = 0
palette_id = 0
ch_input_id = 0
ch_output_id = 0
ch_last_updated = 0
average = 0
samples = 0
aux_a_x = 0
aux_a_y = 0
aux_a_z = 0
aux_b_x = 0
aux_b_y = 0
aux_b_z = 0
color_shift = 0
rotate_vec_x = 0
rotate_vec_y = 0
rotate_vec_z = 0
rotate_vec_s = 1
scale_x = 0
scale_y = 0
scale_z = 0
translate_x = 0
translate_y = 0
translate_z = 0
tag_offset_x = 0
tag_offset_y = 0
tag_offset_z = 0
rotate_rate_x = 0
rotate_rate_y = 0
rotate_rate_z = 0
rotate_x = 0
rotate_y = 0
rotate_z = 0
scale_rate_x = 0
scale_rate_y = 0
scale_rate_z = 0
translate_rate_x = 0
translate_rate_y = 0
translate_rate_z = 0
translate_vec_x = 0
translate_vec_y = 0
translate_vec_z = 0
shader = 0
geometry = 7
line_width = 1
point_size = 0
ratio = 0.1
color_index = 0
color_r = 110
color_g = 110
color_b = 110
color_a = 255
color_fade = 0
texture_id = 0
hide = 0
freeze = 0
topo = 3
facet = 0
auto_zoom_x = 0
auto_zoom_y = 0
auto_zoom_z = 0
trigger_hi_x = 0
trigger_hi_y = 0
trigger_hi_z = 0
trigger_lo_x = 0
trigger_lo_y = 0
trigger_lo_z = 1
set_hi_x = 0
set_hi_y = 0
set_hi_z = 0
set_lo_x = 0
set_lo_y = 0
set_lo_z = 0
proximity_x = 0
proximity_y = 0
proximity_z = 0
proximity_mode_x = 0
proximity_mode_y = 0
proximity_mode_z = 0
segments_x = 16
segments_y = 16
segments_z = 0
tag_mode = 0
format_id = 0
table_id = 0
record_id = id
size = 420

In [65]:
# Person 1 node files
pnode = str(person) + "_" + "temppredict_" + str(viztype) + "_" + str(version) + "_node.csv"
# Person 1 tag files
ptag = str(person) + "_" + "temppredict_" + str(viztype) + "_" + str(version) + "_tag.csv"

## Titles of files

### Change name of dataframe to match which viz version is being run

In [66]:
# Open ANTz Node file for writing
fout = open(pnode,"w")

# Open the Tag file to add metadata to ANTz objects
ftag = open(ptag,"w")
# Write the header string
outputstring = "id,record_id,table_id,title,description\n"
ftag.write(outputstring)

# Write rows for header, world, camera views (4), and grid to Node file
outputstring = "id,type,data,selected,parent_id,branch_level,child_id,child_index,palette_id,ch_input_id,ch_output_id,ch_last_updated,average,samples,aux_a_x,aux_a_y,aux_a_z,aux_b_x,aux_b_y,aux_b_z,color_shift,rotate_vec_x,rotate_vec_y,rotate_vec_z,rotate_vec_s,scale_x,scale_y,scale_z,translate_x,translate_y,translate_z,tag_offset_x,tag_offset_y,tag_offset_z,rotate_rate_x,rotate_rate_y,rotate_rate_z,rotate_x,rotate_y,rotate_z,scale_rate_x,scale_rate_y,scale_rate_z,translate_rate_x,translate_rate_y,translate_rate_z,translate_vec_x,translate_vec_y,translate_vec_z,shader,geometry,line_width,point_size,ratio,color_index,color_r,color_g,color_b,color_a,color_fade,texture_id,hide,freeze,topo,facet,auto_zoom_x,auto_zoom_y,auto_zoom_z,trigger_hi_x,trigger_hi_y,trigger_hi_z,trigger_lo_x,trigger_lo_y,trigger_lo_z,set_hi_x,set_hi_y,set_hi_z,set_lo_x,set_lo_y,set_lo_z,proximity_x,proximity_y,proximity_z,proximity_mode_x,proximity_mode_y,proximity_mode_z,segments_x,segments_y,segments_z,tag_mode,format_id,table_id,record_id,size\n"
fout.write(outputstring)

# Row for world parameters
outputstring = "1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Row for first camera view
outputstring = "2,1,2,0,0,0,2,2,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0.008645,0.825266,-0.564678,1,1,1,-32.446629,-180.908295,143.514175,0,0,1,0,0,0,55.620094,0.600200,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,214.306686,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Row for second camera view
outputstring = "3,1,3,0,2,1,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,1,1,1,-0.500000,0,571.750000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Third camera view
outputstring = "4,1,4,0,2,1,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,-0,1,1,1,0,-90,7,0,0,1,0,0,0,90,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Fourth camera view
outputstring = "5,1,5,0,2,1,5,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,-0,-0,1,1,1,85,0,7,0,0,1,0,0,0,90,270,0,0,0,0,0,0,0,-0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Default Grid
outputstring = "6,6,6,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,3,0,0,255,150,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14,14,0,0,0,0,0,420\n"
fout.write(outputstring)

# Initialize increments
taginc = 0
id = 6

numlevel0toroids = 1
numlevel1toroids = 0

for index, row in dfos1.iterrows() :
    id += 1
    data = id
    type = 5
    parent_id = 0
    selected = 0
    branch_level = 0
    child_id = id
    translate_x = row['hrv_scale']
    translate_y = row['hr_scale']
    translate_z = row['timeaxis']
    scale_x = row['breath_scale']
    scale_y = scale_x
    scale_z = scale_x
    ratio = .1
    line_width= 2
    geometry = 3
    topo = 2
    if row['temp'] <= 35:
        color_r = 0
        color_g = 0
        color_b = 255
    elif (row['temp'] > 35) & (row['temp'] <= 35.5):
        color_r = 93
        color_g = 71
        color_b = 238
    elif (row['temp'] > 35.5) & (row['temp'] <= 36):
        color_r = 42
        color_g = 0
        color_b = 213
    elif (row['temp'] > 36) & (row['temp'] <= 36.5):
        color_r = 111
        color_g = 49
        color_b = 219
    elif (row['temp'] > 36.5) & (row['temp'] <= 37):
        color_r = 99
        color_g = 0
        color_b = 158
    elif (row['temp'] > 37) & (row['temp'] <= 37.5):
        color_r = 153
        color_g = 0
        color_b = 153
    elif (row['temp'] > 37.5) & (row['temp'] <= 38):
        color_r = 161
        color_g = 1
        color_b = 93
    elif (row['temp'] > 38) & (row['temp'] <= 38.5):
        color_r = 241
        color_g = 29
        color_b = 40
    elif (row['temp'] > 38.5) & (row['temp'] <= 39):
        color_r = 253
        color_g = 58
        color_b = 45
    elif (row['temp'] > 39) & (row['temp'] <= 39.5):
        color_r = 254
        color_g = 97
        color_b = 44
    elif (row['temp'] > 39.5) & (row['temp'] <= 40):
        color_r = 255
        color_g = 135
        color_b = 44
    elif (row['temp'] >= 40):
        color_r = 255
        color_g = 161
        color_b = 44
    rotate_x = 0
    rotate_y = 0
    rotate_z = 0
    record_id = id 
    texture_id= 0

    # Creat output string and write to ANTz Node File
    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
    fout.write(outputstring)

    # Output to Tag file
    tagtext = str("Time: ") + str(index) + str("// Temp: ") + str(row['temp']) + str("// RR: ") + str(row['breath'])
    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
    ftag.write(tagstring)
    taginc += 1

    
fout.close()